In [1]:
import pygame
from enum import Enum
import random 

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

(5, 0)

In [3]:
class Direction(Enum):
    LEFT = 0
    UP = 1
    RIGHT = 2
    DOWN = 3

In [4]:
WHITE = (255, 255, 255)
RED = (200,0,0)
BLUE1 = (0, 0, 255)
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPEED = 20

In [64]:
class SnakeGame:
    def __init__(self, w=640, h=480, size=40):
        self.w = w
        self.h = h
        self.display = pygame.display.set_mode((self.w, self.h))
        self.clock = pygame.time.Clock()
        self.clock.tick(200)
        self.size = size
        self.reset()

    def reset(self):
        self.head = (self.w//self.size//2, self.h//self.size//2)
        self.snake = [self.head]

        self.r_direction_index = 2
        self.direction = Direction(self.r_direction_index)
        self.game_over = False
        self.score = 0
        self._place_food()
        self._update_ui()

    def _place_food(self):
        while True:
            x = random.randint(0, (self.w//self.size)-1)
            y = random.randint(0, (self.h//self.size)-1)
            if (x, y) not in self.snake:
                self.food = (x, y)
                break

    def _update_ui(self):
        self.display.fill(BLACK)
        for x in range(self.w//self.size):
            for y in range(self.h//self.size):
                pygame.draw.rect(self.display, (23, 23, 23), pygame.Rect(
                    x*self.size, y*self.size, self.size, self.size), 1)
        for i, (w, h) in enumerate(self.snake):
            if i == 0:
                pygame.draw.rect(self.display, BLUE1, pygame.Rect(
                    w*self.size, h*self.size, self.size, self.size))
                pygame.draw.rect(self.display, BLUE1, pygame.Rect(
                    w*self.size, h*self.size, self.size, self.size), 1)
            else:
                pygame.draw.rect(self.display, BLUE2, pygame.Rect(
                    w*self.size, h*self.size, self.size, self.size))
                pygame.draw.rect(self.display, BLUE2, pygame.Rect(
                    w*self.size, h*self.size, self.size, self.size), 1)
            pass

        pygame.draw.rect(self.display, RED, pygame.Rect(
            self.food[0]*self.size, self.food[1]*self.size, self.size, self.size))
        pygame.display.flip()

    def step(self, action=-1):
        self.reward = 0
        if action != -1:
            if action == 0:  # 왼쪽
                self.r_direction_index -= 1
            elif action == 2:  # 오른쪽
                self.r_direction_index += 1
            self.r_direction_index %= 4
            self.direction = Direction(self.r_direction_index)
            if self.game_over is False:
                self._move(self.direction)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    self.r_direction_index -= 1
                elif event.key == pygame.K_RIGHT:
                    self.r_direction_index += 1
                # elif event.key == pygame.K_UP:
                #     self.direction = Direction.UP
                # elif event.key == pygame.K_DOWN:
                #     self.direction = Direction.DOWN

                self.r_direction_index %= 4
                self.direction = Direction(self.r_direction_index)

                if self.game_over is False:
                    self._move(self.direction)

        # ui 업데이트
        self._update_ui()
        return (self.direction, self.snake[0], (self.food[0], self.food[1])), self.reward, self.game_over

    def _move(self, direcion):
        x, y = 0, 0
        if direcion == Direction.RIGHT:
            x = 1
        elif direcion == Direction.LEFT:
            x = -1
        elif direcion == Direction.UP:
            y = -1
        elif direcion == Direction.DOWN:
            y = 1
        last = self.snake[-1]
        head = self.snake[0]

        self.snake.insert(0, (head[0]+x, head[1]+y))

        is_eat_apple = self._hit_apple()
        is_hit_obstacle = self._hit_obstacles()

        # 장애물이랑 부딪치면?
        if is_hit_obstacle:
            self.reward = -10
            self.game_over = True

        # 사과를 먹으면?
        if is_eat_apple:
            self.reward = 10
            self._place_food()
        else:
            self.snake.pop()

    def _hit_obstacles(self):
        if self.snake[0][0] < 0\
                or self.snake[0][0] > (self.w//self.size-1)\
                or self.snake[0][1] < 0\
                or self.snake[0][1] > (self.h//self.size-1)\
                or self.snake[0] in self.snake[1:]:
            return True
        return False

    def _hit_apple(self):
        if (self.food[0], self.food[1]) == self.snake[0]:
            return True
        else:
            False

    def quit(self):
        pygame.quit()

In [68]:
env = SnakeGame()

(1, 3)


In [71]:
import numpy as np
a = np.array([0,1,0])

(d,b,c), reward, done = env.step(a)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [78]:
a = np.array([0,1,0])

(d,b,c), reward, done = env.step(np.argmax(a))
(d,b,c), reward, done 

((<Direction.RIGHT: 2>, (15, 6), (1, 3)), 0, True)

: 